In [23]:
!pip install import-ipynb
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select

#installation guide for selenium
#https://medium.com/@shanyitan/how-to-install-selenium-and-run-it-successfully-via-jupyter-lab-c3f50d22a0d4

#### regexing

In [51]:
def data_gatherer(scraped_data):
    """ Return (dictionary that contains all the courses per page, canceled courses)"""
    database = dict()
    canceled_courses = []
    for course in scraped_data:
        course_details = []
        info_by_day = dict()
        try:
            name=re.findall(r"(\n[a-z,A-Z]\D{2,}\|)", course)[0].split("|")
            name = re.sub(r"[\n\t]*", "", name[0]).replace("  "," ")

        except:
            name = 'no name'
            
        hours = (re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", course))
        course_id = (re.findall(r"\|\s{3}(\d{5})", course))
        day = (re.findall(r"\n(יום [אבגדהו])", course))
        semester = (re.findall(r"\n(שנתי|סמסטר [אבגדהו])", course))
          
        #checking for canceled courses
        if len(course_id) == 0 or len(hours) == 0:
            if len(hours) == 0:
                canceled_courses.append([course_id,name])
            pass

        else:
            for item in range(len(semester)):
                info_by_day={}
                info_by_day["semester"] = semester[item] 
                try:
                    info_by_day["day"] = day[item]
                except:
                    info_by_day["day"] = 'None'
                try:
                    info_by_day["hours"] = hours[item]
                except:
                    info_by_day["hours"] = 'None' 
                course_details.append(info_by_day)
            database[course_id[0]] = {name:course_details}

    canceled_courses = [x for x in canceled_courses if x != []]
    return database, canceled_courses
database, canceled_courses = data_gatherer(scraped_data)

New regexer

In [26]:
def data_gatherer(scraped_data):
    """ Return (dictionary that contains all the courses per page, canceled courses):
    database = {course_id:{(course_name, lecturer, sug, group): [{semester:, day:, hours:}]}}
    """
    database = dict()
    canceled_courses = []
    
    for course in scraped_data:
        course_details = []
        info_by_day = dict()
        try:
            name=re.findall(r"(\n[a-z,A-Z]\D{2,}\|)", course)[0].split("|")
            name = re.sub(r"[\n\t]*", "", name[0]).replace("  "," ")

        except:
            name = 'no name'
            
        course_id = (re.findall(r"\|\s{3}(\d{5})", course))    
        hours = re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", course)  
        #checking for canceled courses
        if len(course_id) == 0 or len(hours) == 0:
            if len(hours) == 0:
                canceled_courses.append([course_id,name])
            pass
        
        else:
            paragraphs = re.findall(r"\n((?:\n.+)+)", course) #every paragraph supposed to be diffrent sug
            hours, day, semester, sug_lecturer, group = [], [], [], [], []
            for cell in paragraphs:
                hours.append(re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", cell))
                day.append(re.findall(r"\n(יום [אבגדהו])", cell))
                semester.append(re.findall(r"\n(שנתי|סמסטר [אבגדהו])", cell))
                sug_lecturer.append(re.findall(r"(תרג|שות|מטלה|הדר|שעור|מעב|שומ)(.+)", cell))#maybe there is more sugim
                group.append(re.findall(r"(?<=\().?(?=\))", cell))



        

            for i in range(len(hours)):
                for j in range(len(i)):
                info_by_day={}
                try:
                    info_by_day["semester"] = semester[item] 
                except:
                    info_by_day["semester"] = 'None'
                try:
                    info_by_day["day"] = day[item]
                except:
                    info_by_day["day"] = 'None'
                try:
                    info_by_day["hours"] = hours[item]
                except:
                    info_by_day["hours"] = 'None' 
                course_details.append(info_by_day)
                
                try:
                    sug = sug_lecturer[0][0]
                except:
                    print(f"problem with sug in course: {name} {course_id}")
                    sug = "None"
                try:
                    lecturer = sug_lecturer[0][1]
                except:
                    print(f"problem with lecturer in course: {name} {course_id}")
                    lecturer = "None"
                try:
                    group = group[0]
                except:
                    print(f"problem with group in course: {name} {course_id}")
                    group = "None"
                    
            database[course_id[0]] = {(name, lecturer, sug, group[0]):course_details}

    canceled_courses = [x for x in canceled_courses if x != []]
    return database, canceled_courses
database, canceled_courses = data_gatherer(scraped_data)

problem with sug in course: AMIRIM MEETING CLUB מועדון "אמירים"   ['08247']
problem with lecturer in course: AMIRIM MEETING CLUB מועדון "אמירים"   ['08247']
problem with group in course: AMIRIM MEETING CLUB מועדון "אמירים"   ['08247']
problem with sug in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with lecturer in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with group in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with sug in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with lecturer in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with sug in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with lecturer in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with sug in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים   ['35124']
problem with lecturer in course: INDONESIAN FOR BEGINNERS אינדונזית למתחילים

In [88]:
def data_gatherer(scraped_data):
    """ Return (dictionary that contains all the courses per page, canceled courses):
    database = {course_id:{(course_name, lecturer, sug, group): [{semester:, day:, hours:}]}}
    """
    database = dict()
    canceled_courses = []
    
    for course in scraped_data:
        
        course_details = []
        info_by_day = dict()
        try:
            name=re.findall(r"(\n[a-z,A-Z]\D{2,}\|)", course)[0].split("|")
            name = re.sub(r"[\n\t]*", "", name[0]).replace("  "," ")
   
        except:
            name = 'no name'
         
        #finding the constants of the courses
        course_id = (re.findall(r"\|\s{3}(\d{5})", course))#need to improve it
        hours = re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", course)  
        sug_lecturer = (re.findall(r"(תרג|שות|מטלה|הדר|שעור|מעב|שומ)(.+)", course))#maybe there is more sugim
        group = (re.findall(r"(?<=\().?(?=\))", course))
        
        #checking for canceled courses
        if len(course_id) == 0 or len(hours) == 0:
            if len(hours) == 0:
                canceled_courses.append([course_id,name])
            pass
        
        #Find the changing hours within the course
        else:
            paragraphs = re.findall(r"\n((?:\n.+)+)", course) #every paragraph supposed to be diffrent sug
            hours, day, semester = [], [], []
            for cell in paragraphs:
                hours.append(re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", cell))
                day.append(re.findall(r"\n(יום [אבגדהו])", cell))
                semester.append(re.findall(r"\n(שנתי|סמסטר [אבגדהו])", cell))
        
            try:
                sug = sug_lecturer[0][0]
            except:
                print(f"problem with sug in course: {name} {course_id}")
                sug = "None"
            try:
                lecturer = sug_lecturer[0][1]
            except:
                print(f"problem with lecturer in course: {name} {course_id}")
                lecturer = "None"
            try:
                group = group[0]
            except:
                print(f"problem with group in course: {name} {course_id}")
                group = "None"

        
            for i in range(len(hours)):
                info_by_day={}
                for j in range(len(hours[i])):
                    
                    try:
                        info_by_day["semester"] = semester[i][j] 
                    except:
                        info_by_day["semester"] = 'None'
                    try:
                        info_by_day["day"] = day[i][j] 
                    except:
                        info_by_day["day"] = 'None'
                    try:
                        info_by_day["hours"] = hours[i][j] 
                    except:
                        info_by_day["hours"] = 'None' 
                    course_details.append(info_by_day)

                    
            database[course_id[0]] = {(name, lecturer, sug, group):course_details}

    canceled_courses = [x for x in canceled_courses if x != []]
    return database, canceled_courses
database, canceled_courses = data_gatherer(scraped_data)

problem with sug in course: AMIRIM MEETING CLUB מועדון "אמירים"   ['08247']
problem with lecturer in course: AMIRIM MEETING CLUB מועדון "אמירים"   ['08247']


In [49]:
scraped_data = []
with open('text_sample.txt', 'r') as f:
    for line in f:
        content = json.loads(line)
        for c in content:
            scraped_data.append(c)
scraped_data = scraped_data[5:]           

In [81]:
#sorting the courses dict

database_sorted = {}
for key, value in sorted(database.items()): # Note the () after items!
    database_sorted[key] = value

#removing duplicates and sorting removed
dicto = {}
for course in canceled_courses:
    dicto[str(course)] = None
canceled_sorted=sorted((dicto))
print("\n\n number of active courses in updated scraper:",len(database_sorted.keys()))
print("\n\n list of "+str(len(canceled_sorted)))





 number of active courses in updated scraper: 24


 list of 1


In [82]:
print(canceled_courses)

[[[], 'no name']]


In [85]:
# for i in scraped_data:
#     print(i)
#     print("******************")

In [83]:
for course_id, items in database_sorted.items():
    print(course_id, items, sep=", ")

08154, {('The Israeli Palestinian Conflict הקונפליקט הישראלי פלסטיני  ', ' ד"ר אביגיל יעקבסון', 'שות', 'א'): [{'semester': 'סמסטר ב', 'day': 'יום ה', 'hours': '12:15-10:30'}]}
08214, {('ACADEMIC WRITING IN ENGLISH FOR AMIRIM STUDENTS כתיבה אקדמית באנגלית לתלמידי אמירים  ', ' גב חיה פישר', 'תרג', 'א'): [{'semester': 'שנתי', 'day': 'יום ה', 'hours': '14:15-12:30'}]}
08216, {('France: From Rationalism to Realism צרפת: מרציונליזם לריאליזם  ', ' פרופ יואב רינון', 'שות', 'א'): [{'semester': 'סמסטר ב', 'day': 'יום ג', 'hours': '18:15-16:30'}]}
08231, {('THE GOOD AND THE HAPPY: THEMES IN GREEK ETHICS הטובים והמאושרים: סוגיות באתיקה יוונית  ', ' ד"ר נלי טאלר', 'שות', 'א'): [{'semester': 'סמסטר א', 'day': 'יום ג', 'hours': '20:15-18:30'}]}
08240, {('Text and Context - The Biblical Literature ספרות המקרא: טקסט וקונטקסט  ', ' ד"ר רוני גולדשטיין', 'שות', 'א'): [{'semester': 'סמסטר א', 'day': 'יום ג', 'hours': '18:15-16:30'}]}
08242, {('Directed Readings קריאה מודרכת  ', ' גב מעיין רווה', 'תרג', 'א'